## 1. 모듈/라이브러리 임포트

In [1]:
import torch, os
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable

## 2. 하이퍼 파라메터 세팅

In [2]:
max_epoch = 200
batch_size = 100
lr = 0.0002
z_dim = 64
output_dir = './output/'
if not os.path.exists(output_dir) : os.makedirs(output_dir)

## 3. 데이터 준비

In [3]:
# Image processing
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                     std=(0.5, 0.5, 0.5))])
# MNIST dataset
mnist = datasets.MNIST(root='./data/',
                       train=True,
                       transform=transform,
                       download=True)
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

## 4. 모델 정의

In [4]:
# Discriminator
D = nn.Sequential(
    nn.Linear(784, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 1),
    nn.Sigmoid())

# Generator
G = nn.Sequential(
    nn.Linear(z_dim, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 784),
    nn.Tanh())

if torch.cuda.is_available():
    D.cuda()
    G.cuda()

## 5. Loss/Optimizer

In [5]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=lr)
g_optimizer = torch.optim.Adam(G.parameters(), lr=lr)

## 6. 기타 함수

In [ ]:
# [-1,1] -> [0,1]
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

## 7. 트레이닝

In [ ]:
# Start training
total_batch = len(data_loader.dataset)//batch_size
for epoch in range(max_epoch):
    for i, (images, _) in enumerate(data_loader):
        # Build mini-batch dataset
        images = Variable(images.view(batch_size, -1)).cuda()

        # Create the labels which are later used as input for the BCE loss
        real_labels = Variable(torch.ones(batch_size)).cuda()
        fake_labels = Variable(torch.zeros(batch_size)).cuda()

        #============= Train the discriminator =============#
        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = Variable(torch.randn(batch_size, z_dim)).cuda()
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        # Backprop + Optimize
        d_loss = d_loss_real + d_loss_fake
        D.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        #=============== Train the generator ===============#
        # Compute loss with fake images
        z = Variable(torch.randn(batch_size, z_dim)).cuda()
        fake_images = G(z)
        outputs = D(fake_images)

        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)

        # Backprop + Optimize
        D.zero_grad()
        G.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        if (i+1) % 300 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, '
                  'g_loss: %.4f, D(x): %.2f, D(G(z)): %.2f'
                  %(epoch, max_epoch, i+1, total_batch, d_loss.data[0], g_loss.data[0],
                    real_score.data.mean(), fake_score.data.mean()))

    # Save real images
    if (epoch+1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), output_dir+'real_images.png')

    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images.data), output_dir+'fake_images-%d.png' %(epoch+1))

# Save the trained parameters
torch.save(G.state_dict(), './generator.pkl')
torch.save(D.state_dict(), './discriminator.pkl')

/home/wonkonge/anaconda3/envs/kongda/lib/python3.6/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch [0/200], Step[300/600], d_loss: 0.3823, g_loss: 2.7192, D(x): 0.88, D(G(z)): 0.20
Epoch [0/200], Step[600/600], d_loss: 0.3552, g_loss: 2.5309, D(x): 0.86, D(G(z)): 0.17
Epoch [1/200], Step[300/600], d_loss: 0.5208, g_loss: 2.5739, D(x): 0.82, D(G(z)): 0.22
Epoch [1/200], Step[600/600], d_loss: 0.1966, g_loss: 3.6600, D(x): 0.93, D(G(z)): 0.10
Epoch [2/200], Step[300/600], d_loss: 0.1939, g_loss: 3.4554, D(x): 0.92, D(G(z)): 0.06
Epoch [2/200], Step[600/600], d_loss: 0.9577, g_loss: 1.6909, D(x): 0.70, D(G(z)): 0.28
Epoch [3/200], Step[300/600], d_loss: 0.3612, g_loss: 2.2231, D(x): 0.84, D(G(z)): 0.12
Epoch [3/200], Step[600/600], d_loss: 0.4660, g_loss: 2.8302, D(x): 0.81, D(G(z)): 0.11
Epoch [4/200], Step[300/600], d_loss: 0.3124, g_loss: 3.3243, D(x): 0.90, D(G(z)): 0.13
Epoch [4/200], Step[600/600], d_loss: 0.3516, g_loss: 3.3054, D(x): 0.92, D(G(z)): 0.17
Epoch [5/200], Step[300/600], d_loss: 0.3119, g_loss: 2.9956, D(x): 0.92, D(G(z)): 0.14
Epoch [5/200], Step[600/600], d_